<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
import warnings
warnings.simplefilter('ignore')

In [3]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Взглянем на информацию о данных:

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Всего датасет состоит из 5000 записей. Данные содеражат информацию о клиентах страховой компании. Это пол, возраст, зарплата, количество членов семьи клиента и количество страховых выплат за последние 5 лет.

Проверим также данные на наличие явных дубликатов. Неявных дубликатов здесь быть не может, поскольку столбцы содержат лишь числовую информацию

In [5]:
data.duplicated().sum()

153

In [6]:
data = data.drop_duplicates().reset_index(drop=True)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                4847 non-null   int64  
 1   Возраст            4847 non-null   float64
 2   Зарплата           4847 non-null   float64
 3   Члены семьи        4847 non-null   int64  
 4   Страховые выплаты  4847 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 189.5 KB


**Вывод**

Для исследования мы получили датасет содержащий информацию о клиентах страховой компании. Всего датасет состоит из 5000 записей. Данные содеражат информацию о клиентах страховой компании. Это пол, возраст, зарплата, количество членов семьи клиента и количество страховых выплат за последние 5 лет. В данных пропусков не было, но мы нашли дубликаты и удалили их. Текущего объема данных должно хватить для исследования.

## Умножение матриц

**Необходимо ответить на вопрос:**

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
- a. Изменится. Приведите примеры матриц.
- b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Можно предположить что качество линейной регрессии не изменится. Попробуем это доказать:

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Пусть новые данные это $X' = XB$, тогда 

$ w' = ((XB)^T XB)^{-1} (XB)^T y $

$ w' = ((B^T(X^TX)B))^{-1} B^TX^T y $

$ w' = B^{-1}(B^T(X^TX))^{-1} B^TX^T y $

$ w' = (B)^{-1} (X^TX)^{-1}(B^T)^{-1} B^TX^T y $

$ w' = (B)^{-1} (X^TX)^{-1}X^T y $

Поскольку из формулы обучения $w = (X^T X)^{-1} X^T y $ , то подставим $w$ в $w'$

$w' = B^{-1}w$

Отсюда:

$a' = XBw' = XB B^{-1}w = Xw = a$

Таким образом, $a = a'$

**Ответ:** Качество линейной регрессии не изменится

**Обоснование:** $a = a'$

## Алгоритм преобразования

**Алгоритм**

Данные можно преобразовать к другому виду (зашифровать), если умножить матрицу данных на любую обратимую матрицу. Опишем этот алгоритм

Алгоритм:
- генерируем случайную квадратную матрицу, размерность nxn, где n-количество столбцов исходного датасета
- проверяем матрицу на обратимость
- перемножаем матрицу данных на случайную обратимую матрицу
- сравниваем метрики качества у модели обученной на исходных данных и у модели обученной на преобразованных данных

**Обоснование**

Алгоритм не испортит качество линейной регрессии, поскольку при домножении матрицы данных на обратимую матрицу мы доказали, что предсказания не изменятся.

## Проверка алгоритма

Эмпирическая проверка алгоритма: (если новую матрицу умножить на обратную матрицу от рандомной, то должны получить на выходе исходный результат

In [8]:
a = data
matrix_input = np.array(a)
n = matrix_input.shape[1]
random_matrix = np.random.normal(size=(n, n))
matrix_output = matrix_input @ random_matrix
matrix_output

array([[-111222.03613548,  -33897.55268462,   55545.45444019,
          12173.31289793,  -39085.24964448],
       [ -85183.62851585,  -25951.36325873,   42581.19835884,
           9341.78405693,  -29968.12839947],
       [ -47067.49542492,  -14335.80023729,   23538.24731468,
           5167.87099889,  -16567.73817481],
       ...,
       [ -76033.51077563,  -23180.63382225,   37950.49366973,
           8307.9446069 ,  -26699.92843046],
       [ -73339.06108033,  -22356.71812138,   36608.38242219,
           8015.40261119,  -25758.95409604],
       [ -91051.31578314,  -27754.41759934,   45456.66977051,
           9957.8603936 ,  -31983.83799126]])

In [9]:
matrix_output @ (np.linalg.inv(random_matrix))

array([[ 1.00000000e+00,  4.10000000e+01,  4.96000000e+04,
         1.00000000e+00, -9.36532876e-12],
       [-4.21376918e-12,  4.60000000e+01,  3.80000000e+04,
         1.00000000e+00,  1.00000000e+00],
       [ 2.41550119e-12,  2.90000000e+01,  2.10000000e+04,
         2.35565962e-12, -9.28868053e-13],
       ...,
       [-3.11760540e-13,  2.00000000e+01,  3.39000000e+04,
         2.00000000e+00, -1.96287744e-11],
       [ 1.00000000e+00,  2.20000000e+01,  3.27000000e+04,
         3.00000000e+00, -2.51190608e-11],
       [ 1.00000000e+00,  2.80000000e+01,  4.06000000e+04,
         1.00000000e+00, -1.51652677e-11]])

**Получили исходные данные, следовательно алгоритм доказан и теоретически и практически.**

Создадим функцию, которая на вход примет наши данные, преобразует их через матричное умножение этих данных на случайную обратимую матрицу и вернет нам измененные данные.

In [10]:
def generate_inv_matrix(n):
    random_matrix = np.random.normal(size=(n, n))
    try:
        np.linalg.inv(random_matrix)
    except np.linalg.LinAlgError:
        random_matrix = generate_inv_matrix(n)
    return random_matrix
    
def multiplication_algorithm(data):
    matrix_input = np.array(data)
    n = matrix_input.shape[1] # берем размерность которая схлопнется во время умножения
    random_matrix = generate_inv_matrix(n)
    
    matrix_output = matrix_input @ random_matrix
    return pd.DataFrame(matrix_output, index = data.index, columns = data.columns)

In [11]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [12]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

In [13]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=27)

In [14]:
display(features_train.shape)
display(target_train.shape)
display(features_test.shape)
display(target_test.shape)

(3635, 4)

(3635,)

(1212, 4)

(1212,)

In [15]:
model = LinearRegression()
model.fit(features_train, target_train)
predicted_test = model.predict(features_test)
r2 = r2_score(target_test, predicted_test)
print(f'r2 = {r2}')

r2 = 0.41155664787962276


Теперь применим наш алгоритм на данных и смоделируем линейную регрессию еще раз:

In [16]:
new_features = multiplication_algorithm(features)
new_features.head()

,Пол,Возраст,Зарплата,Члены семьи
0,-50369.700276,9646.347282,-32081.403157,83869.466402
1,-38583.394231,7397.576669,-24603.558992,64239.478917
2,-21321.303143,4089.818628,-13603.074320,35497.689708
3,-42351.589636,8103.341826,-26948.071302,70523.447132
4,-26502.889237,5079.157898,-16892.785552,44127.092543


In [17]:
features_train, features_test, target_train, target_test = train_test_split(new_features, target, test_size=0.25, random_state=27)

In [18]:
display(features_train.shape)
display(target_train.shape)
display(features_test.shape)
display(target_test.shape)

(3635, 4)

(3635,)

(1212, 4)

(1212,)

In [19]:
model = LinearRegression()
model.fit(features_train, target_train)
predicted_test = model.predict(features_test)
r2 = r2_score(target_test, predicted_test)
print(f'r2 = {r2}')

r2 = 0.4115566478767706


Значение r2  у моделей одинаковое.

**Вывод**

- В данном проекте необходимо было провести исследование и реализовать алгоритм шифрования данных, не ухудшая качества линейной регрессии. Получив данные мы провели небольшую предобработку, и перешли к изучению алгоритма. 


- Было предположение, что качество модели не изменится, если умножить матрицу данных на случайную обратимую матрицу. Это предположение мы доказали сначала теоретически, а потом и эмпирически.


- Далее мы создали функцию, реализующую алгоритм. Применили ее на нашем датасете и сравнили качество модели по метрике r2. Качество не изменилось.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования